<a href="https://colab.research.google.com/github/JCJG24/Various-Projects/blob/PyTorch_Object_Detection/Copy_of_SNN_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
c!pip install snntorch --quiet

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import itertools
import snntorch as snn
import os
import torch.nn.functional as F

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
#! kaggle datasets download mikoajfish99/lions-or-cheetahs-image-classification
#! unzip lions-or-cheetahs-image-classification
! kaggle competitions download -c dogs-vs-cats
! unzip dogs-vs-cats

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.9/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.9/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open dogs-vs-cats, dogs-vs-cats.zip or dogs-vs-cats.ZIP.


In [ ]:
! unzip test1
! unzip train

In [ ]:
# Dataloader arguments

batch_size = 128
data_path_train = '/content/train'
data_path_test = '/content/test1'

dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Transform definition
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0,), (1,))
])

#mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
#mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

In [ ]:
# Create smaller sample directory
small_train_datapath = '/content/train_small'
#small_test_datapath = '/content/test_small'
os.mkdir(small_train_datapath)
#os.mkdir(small_test_datapath)

In [ ]:
# Fill the smaller sample directory
import shutil
fnames = ['cat.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
  origin = os.path.join(data_path_train, fname)
  destination = os.path.join(small_train_datapath, fname)
  shutil.copyfile(origin, destination)

fnames = ['dog.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
  origin = os.path.join(data_path_train, fname)
  destination = os.path.join(small_train_datapath, fname)
  shutil.copyfile(origin, destination)

In [ ]:
import glob

train_list = glob.glob(os.path.join(small_train_datapath,'*.jpg'))
test_list = glob.glob(os.path.join(data_path_test, '*.jpg'))

In [ ]:
print(len(train_list), len(test_list))

In [ ]:
# Check the Data + Imports
import matplotlib.pyplot as plt
from PIL import Image
random_idx = np.random.randint(1,20000,size=10)

fig = plt.figure()
i=1
for idx in random_idx:
    ax = fig.add_subplot(2,5,i)
    img = Image.open(train_list[idx])
    plt.imshow(img)
    i+=1

plt.axis('off')
plt.show()

In [ ]:
train_list[0].split('/')[-1].split('.')[0]

In [ ]:
int(test_list[0].split('/')[-1].split('.')[0])

In [ ]:
# Create the validation list sampling from the train list
from sklearn.model_selection import train_test_split
train_list, val_list = train_test_split(train_list, test_size=0.2)

In [ ]:
print(len(val_list))

In [ ]:
# Create the custom Dataset
class dataset(torch.utils.data.Dataset):
    def __init__(self,file_list,transform=None):
        self.file_list = file_list
        self.transform = transform
        
        
    #dataset length
    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength
    
    #load an one of images
    def __getitem__(self,idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        
        label = img_path.split('/')[-1].split('.')[0]
        if label == 'dog':
            label=1
            #label = torch.tensor(1, dtype=torch.long)
        elif label == 'cat':
            label=0
            #label = torch.tensor(0, dtype=torch.long)


        return img_transformed,label

In [ ]:
# Create the data for the Dataloaders
train_data = dataset(train_list, transform=transform)
test_data = dataset(test_list, transform=transform)
val_data = dataset(val_list, transform=transform)

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
# Network architecture
num_inputs = 28*28
num_hidden = 1000
num_outputs = 2

#Temporal Dynamics
num_steps = 50
beta = 0.95

In [ ]:
# one of many / simpler
class ConvNet(nn.Module):

  def __init__(self):
    super().__init__()

    # initialize layers
    self.conv1 = nn.Conv2d(1, 64, 3, padding="same")
    self.mxpl1 = nn.MaxPool2d(2)
    self.lif1 = snn.Leaky(beta=beta)

    self.conv2 = nn.Conv2d(64, 128, 3, padding="same")
    self.mxpl2 = nn.MaxPool2d(2)
    self.lif2 = snn.Leaky(beta=beta)

    self.fc = nn.Linear(128 * 7 * 7, num_outputs)
    self.lif3 = snn.Leaky(beta=beta)

  def forward(self, x):

    # initialize hidden states at t=0
    mem1 = self.lif1.init_leaky()
    mem2 = self.lif2.init_leaky()
    mem3 = self.lif3.init_leaky()

    # Record the final layer
    spk3_rec = []
    mem3_rec = []

    # time-loop
    for step in range(num_steps):
      cur1 = self.conv1(x)
      spk1, mem1 = self.lif1(self.mxpl1(cur1), mem1)
      cur2 = self.conv2(spk1)
      spk2, mem2 = self.lif2(self.mxpl2(cur2), mem2)
      cur3 = self.fc(spk2.flatten(1))
      spk3, mem3 = self.lif2(cur3, mem3)

      # store in list
      spk3_rec.append(spk3)
      mem3_rec.append(mem3)

    return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)

In [ ]:
# one of many / more convoluted xixi
class ConvNet2(nn.Module):

    def __init__(self):
        super().__init__()

        # initialize layers
        self.conv1 = nn.Conv2d(3, 64, 3, padding="same")
        self.relu1 = nn.ReLU()
        self.mxpl1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding="same")
        self.relu2 = nn.ReLU()
        self.mxpl2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(128, 256, 3, padding="same")
        self.relu3 = nn.ReLU()
        self.mxpl3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(256 * 3 * 3, 512)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(512, num_outputs)

        # initialize leaky integrate-and-fire activation function
        self.lif1 = snn.Leaky(beta=beta)
        self.lif2 = snn.Leaky(beta=beta)
        self.lif3 = snn.Leaky(beta=beta)
        self.lif4 = snn.Leaky(beta=beta)

    def forward(self, x):

        # initialize hidden state at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        mem4 = self.lif4.init_leaky()

        # Record the final layer
        spk4_rec = []
        mem4_rec = []

        # time-loop
        for step in range(num_steps):
            cur1 = self.conv1(x)
            cur1 = self.relu1(cur1)
            spk1, mem1 = self.lif1(self.mxpl1(cur1), mem1)
            cur2 = self.conv2(spk1)
            cur2 = self.relu2(cur2)
            spk2, mem2 = self.lif2(self.mxpl2(cur2), mem2)
            cur3 = self.conv3(spk2)
            cur3 = self.relu3(cur3)
            spk3, mem3 = self.lif3(self.mxpl3(cur3), mem3)
            cur4 = self.fc1(spk3.flatten(1))
            cur4 = self.relu4(cur4)
            spk4, mem4 = self.lif4(self.fc2(cur4), mem4)

            # store in list
            spk4_rec.append(spk4)
            mem4_rec.append(mem4)

        return torch.stack(spk4_rec, dim=0), torch.stack(mem4_rec, dim=0)

In [ ]:
# Load the network to device
#net = Net().to(device)
net = ConvNet2().to(device)

# Training Parameters
num_epochs = 10
counter = 0
loss_hist = []
test_loss_hist = []

# Define Loss and Optimizer
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnvnet.parameters(), lr=1e-3, betas=(0.9, 0.999))

In [ ]:
# outer training loop -- SPIKE TRAINING LOOP
for epoch in range(num_epochs):
  train_batch = iter(train_loader)


  # Minibatch training loop
  for data, targets in train_batch:
    data = data.to(device)
    targets = targets.to(device)

    # forward pass
    net.train()
    spk_rec, _ = net(data)

    # initialize the loss & sum over time
    loss_val = torch.zeros((1), dtype=dtype, device=device)
    loss_val = loss(spk_rec.sum(0), targets) # batch x num_out

    # Gradient calculation & weight update
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

    # Store loss history for future plotting
    #loss_hist.append(loss_val.item()) ---> forget in colab

    # Print train/test loss/accuracy
    if counter % 10 == 0:
      print(f"Iteration: {counter} \t Train Loss: {loss_val.item()}")
    
    counter += 1
    if counter == 100:
      break

In [ ]:
# Measure Accuracy

def measure_accuracy(model, dataloader):
  with torch.no_grad():
    model.eval()
    running_length = 0
    running_accuracy = 0

    for data, targets in iter(dataloader):
      print(type(targets))

      data = data.to(device)
      targets = targets.to(device)

      #forward pass
      spk_rec, _ = model(data)
      spike_count = spk_rec.sum(0)
      _, max_spike = spike_count.max(1)

      #correct classes for one batch
      num_correct = (max_spike == targets).sum()

      #total accuracy
      running_length += len(targets)
      running_accuracy += num_correct

    accuracy = (running_accuracy / running_length)
    return accuracy.item()

In [ ]:
print(f"Test set accuracy: {measure_accuracy(net, val_loader)}")